In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
### PRE_PROCESSING ###

def load_images_and_create_label(folder, new_pixels):
    
    """
    Loads images from 
    Resize image to 'new_pixels'
    Returns array of images and array of labels
    """
    
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            img = cv2.resize(img, (new_pixels, new_pixels))
            images.append(img)
            if filename.find("cat") == -1:
                # dog 
                labels.append(1)
            else:
                # cat
                labels.append(0)
    return images, labels

In [6]:
def layer_sizes(X, Y):
    
    """
    Defines sizes of input, hidden and output layers
    Returns:
    n_x -- the size of the input layer
    n_h -- the size of the hidden layer
    n_y -- the size of the output layer
    """
    
    n_X = X.shape[0]
    n_Y = Y.shape[0]
    n_H = 4
    
    return (n_X, n_H, n_Y)

In [7]:
def init_parameters(n_X, n_H, n_Y):
    
    """
    Initializes parameters with required shapes
    Returns:
    param - Python dictionary containing parameters:
    W1, b1, W2, b2
    """
    
    W1 = np.random.randn(n_H, n_X) * 0.01
    b1 = np.zeros((n_H, 1))
    W2 = np.random.randn(n_Y, n_H) * 0.01
    b2 = np.zeros((n_Y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [10]:
def sigmoid(z):
    
    """
    Sigmoid non-linearity
    """
    return 1 / (1 + np.exp(-z))

In [13]:
def forward_propogation(X, parameters):
    
    """
    Performs forward propogation.
    Returns:
    cache - Python dictionary conataining values:
    Z1, A1, Z2, A2
    """
    
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]

    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return cache

In [14]:
def compute_cost(A2, Y):
    
    """
    Computes cross-entropy cost.
    """
    
    # number of examples
    m = Y.shape[1]
    
    logprobs = np.multiply(np.log(A2), Y) + np.multiply((1 - Y), np.log(1 - A2))
    cost = logprobs / m
    
    # makes sure cost is the dimension we expect. E.g., turns [[17]] into 17 
    cost = np.squeeze(cost)
    
    return cost

In [16]:
def backward_propogation(X, Y, parameters, cache):
    
    """
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    
    # number of examples
    m = X.shape[1]
    
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    
    A1 = cache["A1"]
    A2 = cahce["A2"]
    
    dZ2 = Y - A2
    dW2 = (1 / m) * np.dot(DZ2, A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.multiply(np.dot(DZ2, W2.T), 1 - np.power(A1, 2))
    dW2 = (1 / m) * np.dot(DZ1, X.T)
    db2 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    
    grad = {"dW2": dW2,
            "db2": db2,
            "dW1": dw1,
            "db1": db1}

In [17]:
def update_parameters(parameters, grads, learning_rate=1.2):
    
    """
    Updates parameters using the gradient descent update 
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    
    W1 = parameters["W1"]
    b1 = paramaters["b1"]
    W2 = parameters["W2"]
    b2 = parametes["b2"]
    
    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]
    
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters